# Importando dados via Pandas

In [1]:
import pandas as pd
dados = pd.read_csv('creditcard.csv')

In [2]:
dados.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


# Analisando a base de dados

In [3]:
# contar a quantidade de linhas que temos (qualquer coluna vale)

n_transacoes = dados['Class'].count()
print("Número de transações {}.".format(n_transacoes))

Número de transações 284807.


In [4]:
# vamos dessas transações, quantas são fraude

n_fraudes = dados['Class'].sum()
n_normais = n_transacoes - n_fraudes
print("Número de fraude é {} e o número de transações normais é {}.".format(n_fraudes,n_normais))

Número de fraude é 492 e o número de transações normais é 284315.


In [5]:
# verificar o percentual de fraude

fraudes_porc = n_fraudes / n_transacoes * 100
normais_porc = n_normais / n_transacoes * 100

print("Número de fraudes: {} ({:.2f} %).".format(n_fraudes,fraudes_porc))
print("Número de normais: {} ({:.2f} %).".format(n_normais,normais_porc))

Número de fraudes: 492 (0.17 %).
Número de normais: 284315 (99.83 %).


# Função para dividir os dados de treino e os dados de teste

In [6]:
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn import tree

def validador(x, y):
  validador = StratifiedShuffleSplit(n_splits=1, test_size=0.1, random_state=0)
  for treino_id, teste_id in validador.split(x, y):
    x_treino, x_teste = x[treino_id], x[teste_id]
    y_treino, y_teste = y[treino_id], y[teste_id]
  return x_treino, x_teste, y_treino, y_teste

def executar_classificador(classificador, x_treino, x_teste, y_treino):
  arvore = classificador.fit(x_treino, y_treino)
  y_pred = arvore.predict(x_teste)
  return y_pred

# Criando a árvore de decisão

In [7]:
# execução do validador
x = dados.drop('Class', axis=1).values
y = dados['Class'].values
x_treino, x_teste, y_treino, y_teste = validador(x, y)

#execução do classificador DecisionTree
classificador_arvore_decisao = tree.DecisionTreeClassifier()
y_pred_arvore_decisao = executar_classificador(classificador_arvore_decisao, x_treino, x_teste, y_treino)

*Árvore de decisão pronta!

## Visualização da árvore de decisão 

In [8]:
import matplotlib.pyplot as plt

def salvar_arvore(classificador, nome):
  plt.figure(figsize=(200,100))
  tree.plot_tree(classificador, filled=True, fontsize=14)
  plt.savefig(nome)
  plt.close()

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

def validar_arvore(y_teste, y_pred):
  print(accuracy_score(y_teste, y_pred))
  print(precision_score(y_teste, y_pred))
  print(recall_score(y_teste, y_pred))
  print(confusion_matrix(y_teste, y_pred))

In [9]:
# criacao da figura da arvore de decisão
# salvar_arvore(classificador_arvore_decisao, "arvore_decisao1.png")

In [10]:
# validaçao da árvore de decisão

validar_arvore(y_teste, y_pred_arvore_decisao)

0.9990871107053826
0.7346938775510204
0.7346938775510204
[[28419    13]
 [   13    36]]


In [11]:
print(classificador_arvore_decisao)
print(classificador_arvore_decisao.get_depth())

DecisionTreeClassifier()
21


In [12]:
%%time

#execução do classificador DecisionTree

classificador_arvore_decisao = tree.DecisionTreeClassifier(max_depth=10, random_state=0)
y_pred_arvore_decisao = executar_classificador(classificador_arvore_decisao, x_treino, x_teste, y_treino)

In [13]:
validar_arvore(y_teste, y_pred_arvore_decisao)

0.9994733330992591
0.9473684210526315
0.7346938775510204
[[28430     2]
 [   13    36]]


*Podemos observar que limitando a 10 níveis de profundidade (max_depht=10) o "Prediction Score" melhorou, saindo de 0.78 e chegando a 0.95. Porém o "Recall score" se manteve.*

*Abaixo tentarei alguns ajustes para ver se melhora esse "Recall Score"*

In [14]:
#execução do classificador DecisionTree
classificador_arvore_decisao = tree.DecisionTreeClassifier(max_depth=10, random_state=0, min_samples_leaf=10)
y_pred_arvore_decisao = executar_classificador(classificador_arvore_decisao, x_treino, x_teste, y_treino)

In [15]:
validar_arvore(y_teste, y_pred_arvore_decisao)

0.9993679997191109
0.8604651162790697
0.7551020408163265
[[28426     6]
 [   12    37]]


*Podemos ver que forçar a árvore ter 10 instâncias por folha (min_samples_leaf=10) fez melhorar o "Recall Score", mas piorou o "Prediction Score"*

In [16]:
#execução do classificador DecisionTree
classificador_arvore_decisao = tree.DecisionTreeClassifier(max_depth=5, random_state=0)
y_pred_arvore_decisao = executar_classificador(classificador_arvore_decisao, x_treino, x_teste, y_treino)

In [17]:
validar_arvore(y_teste, y_pred_arvore_decisao)

0.999403110845827
0.9210526315789473
0.7142857142857143
[[28429     3]
 [   14    35]]


In [18]:
# Executando o Random Forest com o scikit-learn

#%%time
#from sklearn.ensemble import RandomForestClassifier

#classificador_random_forest = RandomForestClassifier(n_estimators=100, random_state=0)
#y_pred_random_forest = executar_classificador(classificador_random_forest, x_treino, x_teste, y_treino)

# Abaixo vamos melhorar esse código

CPU times: user 5min 18s, sys: 378 ms, total: 5min 18s
Wall time: 5min 29s


In [22]:
# Técnica BAGGING
# Executando o Random Forest com o scikit-learn

%%time
from sklearn.ensemble import RandomForestClassifier

classificador_random_forest = RandomForestClassifier(n_estimators=50, random_state=0, max_depth=10)
y_pred_random_forest = executar_classificador(classificador_random_forest, x_treino, x_teste, y_treino)

CPU times: user 1min 30s, sys: 92.9 ms, total: 1min 30s
Wall time: 1min 31s


In [23]:
validar_arvore(y_teste, y_pred_random_forest)

0.9995435553526912
0.9736842105263158
0.7551020408163265
[[28431     1]
 [   12    37]]


In [20]:
salvar_arvore(classificador_random_forest.estimators_[0], "random_forest1.png")
salvar_arvore(classificador_random_forest.estimators_[1], "random_forest2.png")

In [25]:
 %%time
 # Técnica BOOSTING

from sklearn.ensemble import AdaBoostClassifier

classificador_adaboost = AdaBoostClassifier(random_state=0)
y_pred_adaboost = executar_classificador(classificador_adaboost, x_treino, x_teste, y_treino)


CPU times: user 1min 38s, sys: 184 ms, total: 1min 38s
Wall time: 1min 43s


In [27]:
salvar_arvore(classificador_adaboost.estimators_[0],"adaboost1.png")
salvar_arvore(classificador_adaboost.estimators_[1],"adaboost2.png")
validar_arvore(y_teste, y_pred_adaboost)

0.9992626663389628
0.8888888888888888
0.6530612244897959
[[28428     4]
 [   17    32]]


Tivemos:

*   Acurácia : 0.99
*   Precisão : 0.88 (bem menor que o anterior)
*   Recall : 0.65 (resultado abaixo do anterior)




In [28]:
# vamos tentar melhorar colocando o número de estimadores

%%time

classificador_adaboost = AdaBoostClassifier(random_state=0, n_estimators=100)
y_pred_adaboost = executar_classificador(classificador_adaboost, x_treino, x_teste, y_treino)

validar_arvore(y_teste, y_pred_adaboost)

0.999403110845827
0.8636363636363636
0.7755102040816326
[[28426     6]
 [   11    38]]
CPU times: user 2min 52s, sys: 238 ms, total: 2min 52s
Wall time: 2min 52s


In [29]:
# vamos tentar melhorar colocando o número de estimadores

%%time

classificador_adaboost = AdaBoostClassifier(random_state=0, n_estimators=200)
y_pred_adaboost = executar_classificador(classificador_adaboost, x_treino, x_teste, y_treino)

validar_arvore(y_teste, y_pred_adaboost)

0.9995435553526912
0.9285714285714286
0.7959183673469388
[[28429     3]
 [   10    39]]
CPU times: user 5min 43s, sys: 526 ms, total: 5min 44s
Wall time: 5min 44s


------------------------
Pensamento ::

O último resultado do AdaBoost (c/ 200 estimadores) teve um desempenho similar ao RandomForest, porém com tempo de execução maior.

Na melhor execução do RandomForest tivemos **PRECISÃO de 0.97** e **RECALL de 0.75**, porém o AdaBoost teve um valor de **RECALL de 0.77** que é melhor.

Dessa forma, vai depender da prioridade para escolher qual das técnicas vamos utilizar.

Se escolhermos um valor de **PRECISÃO** maior, no caso teremos mais transações detectadas como fraude (deixar passar menos transações que tem fraude). Ou escolhemos um valor de **RECALL** maior, que teremos menos transações bloqueadas sem motivo.

-------------------------- 